<img src="../static/logopython.png" alt="Logo Python" style="width: 300px; display: inline"/>
<img src="../static/deimoslogo.png" alt="Logo Deimos" style="width: 300px; display: inline"/>

# Clase 8: Asincronía en Python

Para empezar, tenemos que distinguir entre _multithreading_ y _multiprocessing_. Las dos imágenes siguientes muestran la diferencia

<img src="../static/multithreading.png" alt="Spyder" style="width: 350px;"/>
<img src="../static/multiprocessing.png" alt="Spyder" style="width: 350px;"/>

En Python podemos gestionar la asincronía con diferentes enfoques

## Multithreading

Mediante el uso del módulo de [threading](https://docs.python.org/3/library/threading.html?highlight=threading)

In [1]:
#!/usr/bin/env python3

from threading import Thread
from time import sleep

def snooze(i):
    print("Hilo {} durmiendo 5 segundos".format(i))
    sleep(5)
    print("Hilo {} despierto".format(i))

def main():
    threads = []

    for i in range(10):
        # Definimos args así para que lo detecte como tupla. Si pusieramos args=(i) lo cogeria como entero
        thread = Thread(target=snooze, args=(i, ))

        # Esto invoca a run
        thread.start()


        threads.append(thread)

    for thread in threads:

        # Esperamos a que terminen los hilos
        thread.join()

if __name__ == "__main__":
    main()


Hilo 0 durmiendo 5 segundos
Hilo 1 durmiendo 5 segundos
Hilo 2 durmiendo 5 segundos
Hilo 3 durmiendo 5 segundos
Hilo 4 durmiendo 5 segundos
Hilo 5 durmiendo 5 segundos
Hilo 6 durmiendo 5 segundos
Hilo 7 durmiendo 5 segundos
Hilo 8 durmiendo 5 segundos
Hilo 9 durmiendo 5 segundos
Hilo 1 despiertoHilo 0 despierto
Hilo 3 despierto
Hilo 6 despierto
Hilo 7 despierto

Hilo 2 despierto
Hilo 4 despierto
Hilo 5 despierto
Hilo 8 despierto
Hilo 9 despierto


<div class="alert alert-danger">Los hilos en Python se crean como hilos nativos del Sistema Operativo dentro del proceso que ejecuta el intérprete de Python. Pero la implementación de referencia de Python, CPython, contiene un mutex llamado GIL (acrónimo de Global Interpreter Lock) que __impide que haya más de un hilo de Python en ejecución al mismo tiempo, ya que el gestor de memoria de CPython no es seguro frente a hilos__. Por tanto, los programas Python __no están preparados para ejecutarse en entornos multihilo__. Más información [aquí](https://wiki.python.org/moin/GlobalInterpreterLock) </div>

## Multiprocessing

Mediante el uso del módulo [multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

In [2]:
#!/usr/bin/env python3

from multiprocessing import Process
from time import sleep

def snooze(i):
    print("Proceso {} durmiendo 5 segundos".format(i))
    sleep(5)
    print("Proceso {} despierto".format(i))

def main():
    processes = []

    for i in range(10):
        process = Process(target=snooze, args=(i, ))
        process.start()
        processes.append(process)

    for process in processes:
        process.join()

if __name__ == "__main__":
    main()


Proceso 0 durmiendo 5 segundos
Proceso 1 durmiendo 5 segundos
Proceso 2 durmiendo 5 segundos
Proceso 3 durmiendo 5 segundos
Proceso 4 durmiendo 5 segundos
Proceso 5 durmiendo 5 segundos
Proceso 6 durmiendo 5 segundos
Proceso 7 durmiendo 5 segundos
Proceso 8 durmiendo 5 segundos
Proceso 9 durmiendo 5 segundos
Proceso 0 despierto
Proceso 1 despierto
Proceso 2 despierto
Proceso 3 despierto
Proceso 4 despierto
Proceso 5 despierto
Proceso 6 despierto
Proceso 8 despierto
Proceso 7 despierto
Proceso 9 despierto


## Multiprocessing con pool de procesos

Mediante el uso del objeto [Pool](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool) del módulo de multiprocessing. Un *Pool* de workers donde ejecutar procesos en paralelo

In [ ]:
#!/usr/bin/env python3

from multiprocessing import Pool
from time import sleep

def snooze(i):
    print("Proceso {} durmiendo 5 segundos".format(i))
    sleep(5)
    print("Proceso {} despierto".format(i))

def main():
    # numero de cpus. Si no le pasamos nada, coge os.cpu_count(), disponible a partir de Python 3.4
    pool = Pool(processes=2)
    results = []

    for i in range(10):
        result = pool.apply_async(snooze, (i, ))
        results.append(result)

    for result in results:
        result.get()

    pool.close()
    pool.join()

if __name__ == "__main__":
    main()


La diferencia entre los dos últimos casos es importante: 

* En el primer caso, tenemos varios procesos corriendo de __manera concurrente__ (10, en concreto). 
* En el segundo caso, tenemos varios procesos corriendo de __manera paralela__ (5, en concreto), pero seguimos teniendo 10 procesos concurrentes.

<div class="alert alert-info">__Concurrencia__ y __paralelismo__ no son lo mismo. Concurrencia es lo que se consigue con el _multitasking_, por ejemplo: una cpu que le va asignando _slots_ de tiempo a varias tareas, y éstas se van alternando. El paralelismo exige que haya físicamente corriendo dos o más tareas al mismo tiempo, por lo que se hace necesario más de una cpu (una por tarea).</div>

Si se ve más fácil, se puede recordar esto:

* _Paralelo_ es lo contrario de _serie_
* _Concurrente_ es lo contrario de _secuencial_

<div class="alert alert-info">Además del módulo *multiprocessing* también existen el módulo [subprocess](https://docs.python.org/3/library/subprocess.html) y [parallelpython](http://www.parallelpython.com/)

## Multithreading y multiprocessing

Se pueden combinar ambas técnicas a la vez

In [ ]:
#!/usr/bin/env python3

from threading import Thread
from multiprocessing import Process
from time import sleep

def snooze(i):
    print("Hilo {} durmiendo 5 segundos".format(i))
    sleep(5)
    print("Hilo {} despierto".format(i))

def thread_snooze():
    threads = []

    for i in range(5):
        thread = Thread(target=snooze, args=(i, ))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

def main():
    processes = []

    for i in range(2):
        process = Process(target=thread_snooze)
        process.start()
        processes.append(process)

    for process in processes:
        process.join()

if __name__ == "__main__":
    main()


## Comunicando procesos

Una manera habitual de comunicar procesos entre sí es mediante una cola

In [ ]:
import multiprocessing
from time import sleep


class MiClase(object):

    def __init__(self, name):
        self.name = name

    def do_something(self):
        proc_name = multiprocessing.current_process().name
        print('Estoy en el proceso {} y me pasan estos datos: {}'.format(proc_name, self.name))
        sleep(5)
        print('Ya he acabado')


def worker(q):
    obj = q.get()
    obj.do_something()


if __name__ == '__main__':
    queue = multiprocessing.Queue()

    p = multiprocessing.Process(target=worker, args=(queue,))
    p.start()


    queue.put(MiClase('Cacahuete'))

    # Cerramos la cola y esperamos al hilo de background, asegurándonos de que los datos se han flusheado
    queue.close()
    queue.join_thread()

    # Esperamos a que termine el proceso
    p.join()


## Corrutinas

Las corrutinas fueron introducidas en Python en la [PEP-342](http://www.python.org/dev/peps/pep-0342/), con Python 3.3. Son un concepto que existe desde hace tiempo en los lenguajes de programación, pero su explicación suele ser oscura.

__Así como los generadores son funciones que usan *yield* para generar datos, las corrutinas son funciones que usan *yield* para consumir datos que les envían desde fuera__

En un generador, la llamada *next(generador)* hace que el generador se ejecute hasta una sentencia *yield*, que devuelve un valor y pausa la función.

En una corrutina, la llamada *corrutina.send(dato)* hace que la corrutina recoja ese dato mediante *yield* y continúe

<div class="alert alert-info">Puedes ver la instrucción *yield* como similar al concepto de [*interrupción*](https://es.wikipedia.org/wiki/Interrupci%C3%B3n) en un sistema operativo: una señal que hace que se deje de ejecutar el código de una función y se pase a ejecutar el de otra</div>

```python
# Generador
for n in range(10):
    yield n # El control pasa a quien esté recorriendo el generador
    
# Corrutina
try:
    while True:
        dato = yield # El control viene de quien me ha hecho el send, y me pasa a mí
        print dato 
        # Haz lo que sea con el dato
except GeneratorExit:
    print("Alguien ha llamado a close()")```

Con este enfoque, se puede simular en Python un sistema de *pipes* como el de UNIX. Unos procesos generan valores, se los envían a otros, estos otros a otros, y así sucesivamente hasta llegar al consumidor final.

In [1]:
def grep(pattern):
    """
        Esto es una corrutina:
        - Al llamarla, le pasamos como argumento el patron a buscar
        - A partir de ahí, cada vez que le mandemos texto con send() desde fuera, lo recogerá con yield
        - Si el texto que le mandemos contiene el patron, se limitara a imprimir la linea recibida
        Estamos implementado un pipe, vamos
    """
    print("Buscando cadena {}".format(pattern))
    try:
        while True:

            # Recojo lo que me manden
            line = (yield)

            # Si el patrón está en lo que me han mandado, lo imprimo
            if pattern in line:
                print(line)
                
            # No tengo porque devolver nada, pero podría hacerlo
            
    except GeneratorExit:
        print("Cerrando")

# Creamos la corrutina. Un filtro para buscar "python" en cadenas que le pasemos
g = grep("python")

# La arrancamos
g.send(None)

# Le mandamos cadenas
g.send("Hola")
g.send("Hola")
g.send("Hola")

# Aqui la encuentra
g.send("python")

# La cerramos
g.close()

Buscando cadena python
python
Cerrando


El concepto de corrutina es lo que subyace en el módulo *asyncio*, como veremos a continuación.

## Asyncio

Si estamos usando _Python 3.4 o superior_, podemos hacer uso del módulo [asyncio](https://docs.python.org/3/library/asyncio.html). Permite la implementación de concurrencia sin necesidad de usar threads ni multiproceso. Se basa en la cooperación entre corrutinas. 

*Asyncio* es simplemente la implementación de un bucle de eventos que simula el comportamiento de un gestor de tareas: una corrutina se ejecuta hasta que encuentra una sentencia *yield from otra_corrutina*. En ese momento, el control pasa a la otra corrutina, que simplemente realiza la operación que tenga que realizar, y devuelve un valor que recoge la función llamante, si le hace falta. Esto sucede con todas las corrutinas que hayan sido registradas en el bucle de eventos hasta que terminan. 

En el ejemplo inferior, *snooze* es una corrutina, y mediante la construcción *yield from*, delega en la corrutina *asyncio.sleep*, que tras un segundo, simplemente regresa. En ese punto, la corrutina *snooze* despierta, y termina. A destacar que, en este caso particular, *snooze* no obtiene ningún valor de la llamada a *sleep*, por eso ni siquiera asigna el resultado de yield from, pero podría hacerlo si quisiera. Tampoco devuelve ningún valor con return, pero también podría hacerlo.

Para que una corrutina sea utilizable como tarea *llamable* en el bucle de eventos, ha de ser decorada mediante *@asyncio.coroutine*. Esto significa que la función ya puede delegar en otras corrutinas mediante *yield from* y ser llamada por otras corrutinas mediante el mismo mecanismo.

In [3]:
#!/usr/bin/env python3
"""
    Ejemplo de uso de asyncio: nos permite implementar concurrencia en un solo hilo de ejecución,
    implementando un bucle de eventos que le va danto turnos a diferentes tareas. Una tarea puede
    detenerse mediante una llamada a yield from. En ese momento, el bucle de eventos le da turno
    a la siguiente tarea
"""
import asyncio

# Con este decorador, marcamos la función como una corrutina utilizable
# por asyncio. Es decir, puede llamar a otras corrutinas con yield from, y ser llamada
# por otras mediante el mismo mecanismo.
@asyncio.coroutine
def snooze(i):
    print("Corrutina {} durmiendo 5 segundos".format(i))

    # En este punto, snooze para durante 5 segundos, hasta que asyncio.sleep vuelve.
    # Si la corrutina devolviera un valor, lo podríamos guardar, pero en este caso simplemente
    # se duerme durante X segundos. 
    yield from asyncio.sleep(5)
    print("Corrutina {} despierta".format(i))

def main():

    # Aquí se construye el bucle de eventos. De entrada, vacío
    loop = asyncio.get_event_loop()
    
    # Las tareas que irán en el bucle
    tasks = []

    for i in range(10):

        # Metemos la tarea en una estructura de datos que nos asegura que se va a mantener viva hasta que la tarea se complete
        # Podría compararse a una promesa en JavaScript
        task = asyncio.ensure_future(snooze(i))
        
        # Añadimos la tarea a la lista
        tasks.append(task)

    # Hacemos que el bucle de eventos se ejecute hasta que terminen todas las tareas
    loop.run_until_complete(asyncio.wait(tasks))
    
    # Cerramos y se acabó
    loop.close()

if __name__ == "__main__":
    main()


Corrutina 0 durmiendo 5 segundos
Corrutina 1 durmiendo 5 segundos
Corrutina 2 durmiendo 5 segundos
Corrutina 3 durmiendo 5 segundos
Corrutina 4 durmiendo 5 segundos
Corrutina 5 durmiendo 5 segundos
Corrutina 6 durmiendo 5 segundos
Corrutina 7 durmiendo 5 segundos
Corrutina 8 durmiendo 5 segundos
Corrutina 9 durmiendo 5 segundos
Corrutina 0 despierta
Corrutina 1 despierta
Corrutina 2 despierta
Corrutina 3 despierta
Corrutina 4 despierta
Corrutina 5 despierta
Corrutina 6 despierta
Corrutina 7 despierta
Corrutina 8 despierta
Corrutina 9 despierta


<div class="alert alert-success">Python 3.4 introdujo el framework *asyncio*. Mediante dicho framework, se implementa un bucle de eventos concurrentes en el que cada tarea a ejecutar es una corrutina decorada mediante @asyncio.coroutine</div>

## Async/await

A partir de Python 3.5, podemos usar [async/await](https://docs.python.org/3/reference/expressions.html#await), que se puede considerar *azúcar sintáctico* con respecto a *asyncio*: Aunque hemos de tener en cuenta lo siguiente_:

* Ya no tenemos que decorar las corrutinas para poder usarlas como tareas .del bucle de eventos. Basta con definirlas con *async* delante de la función. Esta construcción se considera una __corrutina nativa__, en contraste con las corrutinas construídas en base a *yield from*

* En lugar de *yield from*, utilizamos *await*. Aunque técnicamente hay diferencias entre ambas construcciones (await es más restrictiva con lo que acepta como expresión), la idea es la misma: esperar a que otra corrutina termine

In [4]:
#!/usr/bin/env python3

"""
    Esto es básicamente azucar sintáctico sobre asyncio
"""
import asyncio

# Ya no hace falta decorar la función para hacerla una corrutina válida.
# Basta con definirla como async
async def snooze(i):
    print("Corrutina {} durmiendo 5 segundos".format(i))

    # En vez de yield from hacemos await, que es más expresivo. No son exactamente
    # lo mismo, pero la diferencia es sutil. Se explica aquí http://www.snarky.ca/how-the-heck-does-async-await-work-in-python-3-5
    await asyncio.sleep(5)

    print("Corrutina {} despierta".format(i))


def main():

    loop = asyncio.get_event_loop()
    tasks = []

    for i in range(10):
        task = snooze(i)
        tasks.append(task)

    loop.run_until_complete(asyncio.wait(tasks))
    loop.close()


if __name__ == "__main__":
    main()


RuntimeError: Event loop is closed

##### <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es"><img alt="Licencia Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Curso Python</span> por <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Jorge Arévalo</span> se distribuye bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es">Licencia Creative Commons Atribución 4.0 Internacional</a>.

---
_Las siguientes celdas contienen configuración del Notebook_

_Para visualizar y utlizar los enlaces a Twitter el notebook debe ejecutarse como [seguro](http://ipython.org/ipython-doc/dev/notebook/security.html)_

    File > Trusted Notebook

In [4]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_file = '../static/styles/style.css'
HTML(open(css_file, "r").read())